# **Reti Neurali** 

## **Esercizio 1: Download e pre-processamento dei dati.**

Scaricare e pre-processare i dati per il successivo addestramento del modello. 

Il dataset che utilizzeremo sarà CIFAR10, scaricabile dalla libreria `tensorflow.keras.datasets`

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Download dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = y_train.ravel()
y_test = y_test.ravel()

# Stampare le shape dei dati
print('dati di train:',x_train.shape)
print('dati di test:',x_test.shape)

dati di train: (50000, 32, 32, 3)
dati di test: (10000, 32, 32, 3)


In [17]:
from sklearn.preprocessing import StandardScaler

# Pre-processamento dei dati
scaler=StandardScaler()

x_train = x_train.reshape(x_train.shape[0], -1)  # Flatten the images
x_test = x_test.reshape(x_test.shape[0], -1)  # Flatten the images
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


## **Esercizio 2: Creare modello MLP**

Per creare il modello MLP utilizziamo l' oggetto `MLPClassifier` dal modulo `sklearn.neural_networks`. Questo è un oggetto molto complesso che prevede la possibilità di specificare tanti parametri, permettendoci una personalizzazione molto dettagliata. Vediamo di seguito gli argomenti principali:

- `hidden_layer_sizes`: rappresenta la struttura dell' MLP, sotto forma di una tupla. La tupla deve essere composta da numeri interi, ogni numero indica il numero di neuroni presenti nel rispettivo layer.

Esempio: 

`hidden_layer_sizes` = `(100)`

creerà un solo layer con 100 neuroni

`hidden_layer_sizes` = `(100, 50)`

creerà due layer, il primo con 100 neuroni, il secondo invece con 50.

- `max_iter`: massimo numero di iterazioni per raggiungere la convergenza. 

- `activation`: indica quale funzione di attivazione utilizzare, valori possibili sono `'relu'`, `'logistic'`, `'tanh'` and `'identity'`.

- `solver`: indica quale algoritmo di ottimizzazione utilizzare, valori possibili sono `'adam'`, `'sgd'` and `'lbfgs'`.

- `learning_rate_init`: valore iniziale del learning rate.

- `verbose`: valore booleano che, se impostato su `True`, stampa l' output di ogni iterazione di training. Molto utile per monitorare il training.

- `random_state`: fissa il seed della randomizzazione.


Per iniziare creiamo un MLP molto basilare, alleniamolo e testiamone le performance. Come parametri utilizzeremo:

- `hidden_layer_sizes` = `(100)`

- `max_iter` = `20`

- `random_state` = `42`

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP 
MLP=MLPClassifier(hidden_layer_sizes = (100),max_iter = 20,random_state = 42)

# Allenare MLP
MLP.fit(x_train,y_train)

# Valutare MLP
predictions_MLP=MLP.predict(x_test)

# Stampare l' accuratezza
accuracy_MLP = accuracy_score(y_test, predictions_MLP)
print('Accuracy:',accuracy_MLP)

Accuracy: 0.5057


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


## **Esercizio 2.1: Aumentiamo i parametri del nostro modello**

Proviamo adesso ad aumentare i dettagli del nostro modello, modificando o aggiungendo i parametri sopra specificati. 

In [33]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP con più strati e altre specifiche
MLP_1=MLPClassifier(hidden_layer_sizes = (100,50),max_iter = 1,random_state = 42,activation='relu',solver='sgd',warm_start=True)

## **Esercizio 3: Implementare manualmente l' algoritmo di early stopping.**

L' algoritmo di early stopping ci permette di terminare anticipatamente l' allenamento di un modello nel caso in cui questo raggiunga la convergenza. Supponiamo infatti che il nostro modello raggiunga un certo livello di accuratezza e che non riesca a migliorare oltre quel livello. Questo significa che il modello, da quel momento in poi, non sta più apprendendo nuove informazioni, per cui le successive iterazioni sono superflue, ed inoltre rischiano di essere dannose, spingendo il modello verso l' overfitting. 

L' early stopping verifica ad ogni iterazione che l' accuratezza del modello sia incrementata di una certa tolleranza. Se questa tolleranza non viene superata per un certo numero di epoche, allora possiamo decidere di stoppare l' allenamento in quanto il modello ha raggiunto la convergenza.

**N.B: per applicare early stopping è necessario specificare i seguenti parametri dell' MLP:**

- `warm_start`=`True` in modo che il training proceda dallo stato attuale del modello e non dall' inizializzazione.

- `max_iter`=`1` in modo che il modello venga allenato per una sola epoca. Per l' early stopping infatti dovremo gestire manualmente il numero di iterazioni.

In [46]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


#divisione del train set in train set e validation set
validation_size = int(0.2 * len(x_train))
x_val = x_train[:validation_size]
y_val = y_train[:validation_size]
x_train_split = x_train[validation_size:]
y_train_split = y_train[validation_size:]

n_total_epochs = 100  
patience = 10         
tolerance = 1e-4      

best_test_accuracy = 0.0
epochs_without_improvement = 0

for epoch in range(n_total_epochs):
    MLP_1.fit(x_train, y_train)

    y_val_pred = MLP_1.predict(x_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)

    print(f"Epoca {epoch + 1}/{n_total_epochs}, Accuracy: {val_accuracy:.4f}")

    if val_accuracy >best_test_accuracy -tolerance:
        best_test_accuracy = val_accuracy
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print("Arresto anticipato")
        break

Epoca 1/100, Accuracy: 0.8828
Epoca 2/100, Accuracy: 0.8835
Epoca 3/100, Accuracy: 0.8836
Epoca 4/100, Accuracy: 0.8854
Epoca 5/100, Accuracy: 0.8838
Epoca 6/100, Accuracy: 0.8865
Epoca 7/100, Accuracy: 0.8851
Epoca 8/100, Accuracy: 0.8880
Epoca 9/100, Accuracy: 0.8865
Epoca 10/100, Accuracy: 0.8883
Epoca 11/100, Accuracy: 0.8866
Epoca 12/100, Accuracy: 0.8894
Epoca 13/100, Accuracy: 0.8900
Epoca 14/100, Accuracy: 0.8889
Epoca 15/100, Accuracy: 0.8898
Epoca 16/100, Accuracy: 0.8911
Epoca 17/100, Accuracy: 0.8893
Epoca 18/100, Accuracy: 0.8903
Epoca 19/100, Accuracy: 0.8932
Epoca 20/100, Accuracy: 0.8896
Epoca 21/100, Accuracy: 0.8916
Epoca 22/100, Accuracy: 0.8892
Epoca 23/100, Accuracy: 0.8933
Epoca 24/100, Accuracy: 0.8930
Epoca 25/100, Accuracy: 0.8935
Epoca 26/100, Accuracy: 0.8931
Epoca 27/100, Accuracy: 0.8944
Epoca 28/100, Accuracy: 0.8969
Epoca 29/100, Accuracy: 0.8968
Epoca 30/100, Accuracy: 0.8976
Epoca 31/100, Accuracy: 0.8984
Epoca 32/100, Accuracy: 0.8996
Epoca 33/100, Acc